In [1]:
from google.colab import drive
#drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

In [2]:
!pip install estnltk==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.5 MB/s eta 0:00:00


In [3]:
!pip install evaluate
!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Pr

In [4]:
# Impordid
import os
import numpy as np
import pandas as pd
from collections import defaultdict
import evaluate
from datasets import Dataset, DatasetDict

# Korpuse lugemiseks
from estnltk.converters.conll import conll_importer

# _
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AdamWeightDecay
from transformers import DataCollatorForTokenClassification
from transformers import create_optimizer
from transformers.keras_callbacks import KerasMetricCallback
from transformers import TFAutoModelForTokenClassification

In [5]:
def get_dataset_paths(dataset: str) -> dict:
    # Sisend: andmestiku nimi sõnena (ewt/edt)
    # Väljund: sõnastik, mis sisaldab train/dev/test failiteid
    dataset_dir = os.path.join('data', dataset)
    files = os.listdir(dataset_dir)

    paths = {}
    for split in ['train', 'dev', 'test']:
        matching_file = next(f for f in files if f'-ud-{split}.' in f)
        paths[split] = os.path.join(dataset_dir, matching_file)

    return paths

edt = get_dataset_paths('edt')
ewt = get_dataset_paths('ewt')
print(edt)
print(ewt)

{'train': 'data/edt/et_edt-ud-train.conllu', 'dev': 'data/edt/et_edt-ud-dev.conllu', 'test': 'data/edt/et_edt-ud-test.conllu'}
{'train': 'data/ewt/et_ewt-ud-train.conllu', 'dev': 'data/ewt/et_ewt-ud-dev.conllu', 'test': 'data/ewt/et_ewt-ud-test.conllu'}


In [6]:
def preprocess(dataset_path: str) -> list:
    # Sisend: andmestiku failitee sõnena
    # Väljund: List, mis sisaldab parsitud lauseid
    # Iga lause on paaride list kujul [(w0, t0), (w1, t1), ..., (wn, tn)], kus w tähistab sõna ja t sõnale vastavat märgendit.

    dataset = conll_importer.conll_to_text(file=dataset_path)
    parsed_sents = []
    known_tags = ['B-Eve', 'B-Gep', 'B-Loc', 'B-Muu', 'B-Org', 'B-Per', 'B-Prod', 'B-Unk', 'I-Eve', 'I-Gep', 'I-Loc', 'I-Muu', 'I-Org', 'I-Per', 'I-Prod', 'I-Unk']

    # Kuna andmestikus on üksikud vead, aga on enam-vähem selge, mida tegelikult mõeldi, siis teeme vastavad parandused.
    corrections = {
      'B-OrgSpaceAfter': 'B-Org',
      'B_Gep': 'B-Gep',
      'i-Prod': 'I-Prod',
      'Org': 'B-Org',
      'Per': 'B-Per',
      'BäOrg': 'B-Org',
      'B.Prod': 'B-Prod',
      'I-per': 'I-Per'
    }

    for sent in dataset.sentences:
        parsed_sent = []
        for word, misc in zip(sent.words, sent.conll_syntax.misc):
            tag = 'O'
            if misc:
                if 'NE' in misc:
                  if misc['NE'] in known_tags:
                    tag = misc['NE']
                  else:
                    # Kaks üksikut juhtu, kus kahe elemendi pikkune nimeüksus oli märgendatud (_, Per), (_, Per) või (_, Org), (_, Org)
                    if parsed_sent[-1][1] == 'B-Org' and misc['NE'] == 'Org':
                      tag = 'I-Org'
                    if parsed_sent[-1][1] == 'B-Per' and misc['NE'] == 'Per':
                      tag = 'I-Per'
                    else:
                      tag = corrections[misc['NE']]
            pair = (word.text, tag)
            #print(f"({word.text}, {tag})")
            parsed_sent.append(pair)
        parsed_sents.append(parsed_sent)

    return parsed_sents

In [7]:
ewt_dev_sents = preprocess(ewt['dev'])
ewt_train_sents = preprocess(ewt['train'])
ewt_test_sents = preprocess(ewt['test'])

#edt_dev_sents = preprocess(edt['dev'])
#edt_train_sents = preprocess(edt['train'])
#edt_test_sents = preprocess(edt['test'])

print(ewt_dev_sents[0])

[('täiesti', 'O'), ('nõus', 'O'), ('.', 'O')]


In [8]:
# https://github.com/Kyubyong/nlp_made_easy/blob/master/Pos-tagging%20with%20Bert%20Fine-tuning.ipynb
all_tags = ['O','B-Eve', 'B-Gep', 'B-Loc', 'B-Muu', 'B-Org', 'B-Per', 'B-Prod', 'B-Unk', 'I-Eve', 'I-Gep', 'I-Loc', 'I-Muu', 'I-Org', 'I-Per', 'I-Prod', 'I-Unk']
tag2idx = {tag:idx for idx, tag in enumerate(all_tags)}
idx2tag = {idx:tag for idx, tag in enumerate(all_tags)}
# Sõnastikud, kus on vastavuses arv:märgend ja vastupidi, näiteks tag2idx sõnastikus 'B-Eve' -> 1 ning idx2tag sõnastikus siis 1 -> 'B-Eve'

In [9]:
def split_to_token_and_tag(sents, tag2idx):
  # Sisend: parsitud laused ja tag2idx sõnastik
  # Väljund: Sõnastike list
  # Sõnastik sisaldab kolme elementi: lause ID täisarvuna, märgendite list arvulisel kujul ning sõnade list

  res = {}
  #res = []
  for i, sent in enumerate(sents):
    tags = [tag2idx[tag] for _, tag in sent]
    words = [word for word, _ in sent]
    res[i] = {
        'id': i,
        'tags': tags,
        'tokens': words
    }

  return res

def transform_set(data):
  transformed = {
      "id": [v["id"] for v in data.values()],
      "tags": [v["tags"] for v in data.values()],
      "tokens": [v["tokens"] for v in data.values()]
  }
  ds = Dataset.from_dict(transformed)
  return ds

def process_all(train_sents, dev_sents, test_sents, tag2idx):
  # Sisend: train/dev/test lausete listid ja tag2idx sõnastik
  # Väljund: töödeldud andmestik
  train = split_to_token_and_tag(train_sents, tag2idx)
  dev = split_to_token_and_tag(dev_sents, tag2idx)
  test = split_to_token_and_tag(test_sents, tag2idx)

  train_ds = transform_set(train)
  dev_ds = transform_set(dev)
  test_ds = transform_set(test)

  dataset = DatasetDict({
      'train': train_ds,
      'dev': dev_ds,
      'test': test_ds
  })

  return dataset

# dev_split = split_to_token_and_tag(ewt_dev_sents, tag2idx)
# transformed_dev = {
#     "id": [v["id"] for v in dev_split.values()],
#     "tags": [v["tags"] for v in dev_split.values()],
#     "tokens": [v["tokens"] for v in dev_split.values()]
# }
# ds = Dataset.from_dict(transformed_dev)

In [10]:
ewt_dataset = process_all(ewt_train_sents, ewt_dev_sents, ewt_test_sents, tag2idx)
#edt_dataset = process_all(edt_train_sents, edt_dev_sents, edt_test_sents, tag2idx)

In [11]:
print(ewt_dataset)
print(ewt_dataset['dev'][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'tags', 'tokens'],
        num_rows: 5444
    })
    dev: Dataset({
        features: ['id', 'tags', 'tokens'],
        num_rows: 833
    })
    test: Dataset({
        features: ['id', 'tags', 'tokens'],
        num_rows: 913
    })
})
{'id': 0, 'tags': [0, 0, 0], 'tokens': ['täiesti', 'nõus', '.']}


In [12]:
tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT", max_length=128, padding="max_length", truncation=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/410k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
# https://huggingface.co/docs/transformers/en/tasks/token_classification

def tokenize_and_align_labels(sent):
  # Sisend: üks lause
  # Väljund: tokenizeri abil sõnestatud lause, sõnastik sisaldab input_ids, token_type_ids, attention_mask ja labels
  # input_ids on lause 'arvulisel' kujul, igale ID-le vastab mingi sõna (teisendamine funktsiooni tokenizer.convert_ids_to_tokens(input_ids) abil)
  # labels on märgendite list, kus märgendid kattuvad tokenizeri abil sõnestatud lausega

  tokenized_inputs = tokenizer(sent['tokens'], truncation=True, is_split_into_words=True, max_length=128, padding="max_length")
  labels = []
  word_ids = tokenized_inputs.word_ids()
  prev_word_idx = None
  for word_idx in word_ids:
    if word_idx is None:
      labels.append(-100)
    elif word_idx != prev_word_idx:
      labels.append(sent['tags'][word_idx])
    else:
      labels.append(-100)
    previous_word_idx = word_idx

  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [14]:
example = ewt_dataset['train'][1]
print(f'Lause algselt: {example}')

test_align = tokenize_and_align_labels(example)
tokens = tokenizer.convert_ids_to_tokens(test_align.input_ids)
print(f'Lause nö arvulisel kujul: {test_align.input_ids}')
# Osad sõned tükeldatakse veel, näiteks 'reetpa67' -> 'reet', '##pa', '##67'
print(f'Sõnestatud: {tokens}')
aligned_labels = test_align['labels']
print(f'Märgendid paigas: {aligned_labels}')

Lause algselt: {'id': 1, 'tags': [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tokens': ['reetpa67', ':', 'Juba', 'teist', 'aastat', 'ei', 'õitse', 'mul', 'aias', 'nartsissid', '.']}
Lause nö arvulisel kujul: [2, 21999, 199, 23498, 137, 368, 1325, 887, 82, 6917, 164, 392, 10013, 3553, 3188, 379, 15, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sõnestatud: ['[CLS]', 'reet', '##pa', '##67', ':', 'juba', 'teist', 'aastat', 'ei', 'oi', '##tse', 'mul', 'aias', 'nar', '##tsis', '##sid', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [179]:
# NB! Kasutades DatasetDict ja Dataset on see siin ebavajalik, saab otse dataset.map() kasutada
def tokenize_dataset(dataset_dict):
    # Sisend: andmestik sõnastikuna, mis sisaldab train/dev/test hulkasid
    # Väljund:
    processed_dataset = {}

    for split_name, split_data in dataset_dict.items():
        #examples = [split_data[idx] for idx in range(len(split_data))]
        #print(examples == split_data)
        #tokenized_dataset = list(map(tokenize_and_align_labels, examples))
        tokenized_dataset = list(map(tokenize_and_align_labels, split_data))
        processed_split = {
            'input_ids': [],
            'attention_mask': [],
            'labels': []
        }

        for item in tokenized_dataset:
            processed_split['input_ids'].append(item['input_ids'])
            processed_split['attention_mask'].append(item['attention_mask'])
            processed_split['labels'].append(item['labels'])

        processed_dataset[split_name] = processed_split

    return processed_dataset

In [15]:
#tokenized_ewt = tokenize_dataset(ewt_dataset)
#tokenized_edt = tokenize_dataset(edt_dataset)

tokenized_ewt = ewt_dataset.map(tokenize_and_align_labels)
#tokenized_edt = edt_dataset.map(tokenize_and_align_labels)

#print(tokenized_ewt['train']['input_ids'][1])
#print(tokenized_ewt['train']['labels'][1])
#print(tokenizer.convert_ids_to_tokens(tokenized_ewt['train']['input_ids'][1]))

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

In [18]:
print(tokenized_ewt['train'])

Dataset({
    features: ['id', 'tags', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 5444
})


In [19]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")
seqeval = evaluate.load("seqeval")

In [20]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [all_tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_tags[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [21]:
batch_size = 8
num_train_epochs = 1
num_train_steps = (len(tokenized_ewt["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [22]:
model = TFAutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", num_labels=len(all_tags), id2label=idx2tag, label2id=tag2idx)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# train_ewt = Dataset.from_dict({
#         'input_ids': np.array(tokenized_ewt['train']['input_ids']),
#         'attention_mask': np.array(tokenized_ewt['train']['attention_mask']),
#         'labels': np.array(tokenized_ewt['train']['labels'])
#     })
# dev_ewt = Dataset.from_dict({
#         'input_ids': np.array(tokenized_ewt['dev']['input_ids']),
#         'attention_mask': np.array(tokenized_ewt['dev']['attention_mask']),
#         'labels': np.array(tokenized_ewt['dev']['labels'])
#     })
# test_ewt = Dataset.from_dict({
#         'input_ids': np.array(tokenized_ewt['test']['input_ids']),
#         'attention_mask': np.array(tokenized_ewt['test']['attention_mask']),
#         'labels': np.array(tokenized_ewt['test']['labels'])
#     })

# tf_train_set = model.prepare_tf_dataset(
#     train_ewt,
#     shuffle=True,
#     batch_size=8,
#     collate_fn=data_collator,
# )

# tf_validation_set = model.prepare_tf_dataset(
#     dev_ewt,
#     shuffle=False,
#     batch_size=8,
#     collate_fn=data_collator,
# )

tf_train_set = model.prepare_tf_dataset(
    tokenized_ewt['train'],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_ewt['dev'],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

In [24]:
model.compile(optimizer=optimizer)

In [25]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback]

In [26]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=1, callbacks=callbacks)

680/680 [==============================] - 208s 275ms/step - loss: 0.2489 - val_loss: 0.3237 - precision: 0.7263 - recall: 0.6734 - f1: 0.6989 - accuracy: 0.9415


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
